## Creating Tables in a Database

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.connect('planets.db')
cur = conn.cursor()

In [ ]:
cur.execute("""
CREATE TABLE planets (
    id INTEGER PRIMARY KEY,
    name TEXT,
    color TEXT,
    num_of_moons INTEGER,
    mass REAL
);
""")
cur.execute("""ALTER TABLE planets ADD COLUMN rings BOOLEAN;""")
cur.execute("""
INSERT INTO 
    planets 
        (name, color, num_of_moons, mass, rings) 
    VALUES 
        ("Mercury", "gray", 0, 0.55, 0),
        ("Venus", "yellow", 0, 0.82, 0),
        ("Earth", "blue", 1, 1.00, 0),
        ("Mars", "red", 2, 0.11, 0),
        ("Jupiter", "orange", 53, 317.90, 0),
        ("Saturn", "hazel", 62, 95.19, 1),
        ("Uranus", "light blue", 27, 14.54, 1),
        ("Neptune","dark blue", 14, 17.15, 1),
        ("Pluto", "brown", 5, 0.003, 0);
""")

In [ ]:
cur.execute("""
CREATE TABLE exoplanets (
    id INTEGER PRIMARY KEY,
    name TEXT,
    color TEXT,
    num_of_moons INTEGER,
    mass REAL
);
""")
cur.execute("""ALTER TABLE exoplanets ADD COLUMN rings BOOLEAN;""")
cur.execute("""
INSERT INTO 
    exoplanets 
        (name, color, num_of_moons, mass, rings) 
    VALUES
    ("Xya", "green", 4, 3.21, 1),
    ("Loas", "blue", 1, 1.90, 0),
    ("Dsfa", "yellow", 23, 12.82, 2),
    ("Pesa", "red", 0, 0.90, 0);
""")

## Exploring the Database

In [ ]:
# List of tables
cur.execute("select name from sqlite_master where type = 'table';").fetchall()

In [ ]:
cur.execute('''
  select * from exoplanets
''').fetchall()


In [ ]:
cur.execute('''
  select 
    planets.name

  from 
    planets

''').fetchall()

In [ ]:
print([description[0] for description in cur.description])


### Easier seeing full result


In [ ]:
def sql_with_cols(query,cursor=cur):
  '''
  Gives me the full result (with columns)
  '''
  result = cursor.execute(query).fetchall()
  cols = tuple([description[0] for description in cur.description])
  
  full_result = [cols] + result[:] 
  return full_result

In [ ]:
sql_with_cols('select * from exoplanets')

## More Exploration

In [ ]:
results = sql_with_cols('''
SELECT 
   exoplanets.name,
   exoplanets.color,
   exoplanets.num_of_moons 
FROM 
   exoplanets

''')

display(results)

In [ ]:
results = cur.execute('''
SELECT 
   exoplanets.name,
   exoplanets.color,
   exoplanets.num_of_moons 
FROM 
   exoplanets
''')

# df = pd.DataFrame(columns=results[0], data=results[1:])
# df.head()
results = results.fetchall()
df = pd.DataFrame(results, columns=[description[0] for description in cur.description])
df.head()

In [ ]:
sql_with_cols('''
SELECT 
   *
FROM 
   planets 
''')

In [ ]:
sql_with_cols('''
SELECT 
   num_of_moons, 
   count(*) 
FROM 
   planets 
GROUP BY
   num_of_moons
''')